<a href="https://colab.research.google.com/github/Rusildur/Pipeline_module/blob/main/pipeline_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Расчетный модуль для трубопроводов

## Исходные данные для всех расчетов и допущения

In [ ]:
from math import *
import numpy as np
import pandas as pd 
from openpyxl import * 

Авторизуемся для соединения Colab c Drive для чтения sheets

Таблицы с сведениями по веществам

https://docs.google.com/spreadsheets/d/1eOAJ_aZ6n5LLz7yxdIRQMLoxkY5uZdpgfizijixzC0M/edit?usp=sharing


In [ ]:
from google.colab import auth
import gspread
from google.auth import default
autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
wb = gc.open('pipeline_bd')

In [ ]:
rows_agent = wb.sheet1.get_all_values()

In [ ]:
df_agents = pd.DataFrame(rows_agent[1:], columns=rows_agent[0])

In [ ]:
df_agents.head()

In [ ]:
P_0 = 101000  #атмосферное давление ПА
ro = 890  # плотность нефти кг/м3   ## тестово данные подаются вручную, дальше будет разработана функция выбора вещества из БД
ro = ro/1000

###Частота иницирующих событий

Выбор значений частот возникновения событий, инициирующих аварии произведен на основе обобщенных статистических данных и мнения экспертов. Учитывая, что в настоящее время отсутствует нормализованный механизм по сбору статистики отказов оборудования, при использовании статистических данных из литературных источников следует оценивать степень их достоверности и понимать, что такие данные, как правило, дают лишь порядок величины. 

В данной работе при оценке риска используются сведения, рекомендованные приказом Ростехнадзора № 387 от 03.11.2022 
https://drive.google.com/file/d/1VJbsNr1Mxuxcyt2jCvMhBAPyHM8Lqheh/view?usp=share_link

In [ ]:
wb_freq = gc.open("pipeline_bd").get_worksheet(2)

In [ ]:
rows_freq = wb_freq.get_all_values()

In [ ]:
df_freq = pd.DataFrame(rows_freq[1:], columns=rows_freq[0])

In [ ]:
df_freq.head()

In [ ]:
df_freq['частичная'] = df_freq['частичная'].str.replace(',', '.').astype(float)
df_freq['полная'] = df_freq['полная'].str.replace(',', '.').astype(float)

In [ ]:
df_freq

In [ ]:
#df_freq = pd.DataFrame({'оборудование':['труба_75', 'труба_75_150', 'труба_150	'],                  'частичная':[1.0e-06,3.0e-07,1.000000e-07],                 'полная':[5.e-06, 2.000000e-06, 5.000000e-07]                  })

In [ ]:
df_freq

,оборудование,частичная,полная
0,труба_75,1.000000e-06,5.000000e-06
1,труба_75_150,3.000000e-07,2.000000e-06
2,труба_150\t,1.000000e-07,5.000000e-07


### Вероятность возникновения сценариев аварий (test)

In [ ]:
scenario_lvz = {'полная': {
                'пролив': 0.608, 
                'пожар': 0.2,
                'вспышка': 0.0768,
                'ТВС': 0.1152}, 
                'частичная': {
                'пролив': 0.93026, 
                'пожар': 0.035,
                'вспышка': 0.026402,
                'ТВС': 0.0083376}}                 

# Функции для обработки данных

##Подбора веществ

Для поиска в датафрейме/сводной таблице/базе данных сведений по веществу

Входные параметры: вещество - "agent" и оборудование -  "facilities".

In [ ]:
def choice_agent():
  pass 


##Объем трубопровода


In [ ]:
def Volume_pipeline(dl, diam, st):
    """"Функция расчета объема трубопровода/
    Принимает длину в метрах, диаметр и толщину стенки в мм"""
    radius = (diam/2 - st)/1000     #находим радиус и переводим в метры
    V = pi * dl * radius ** 2
    return V

##Площадь пролива

In [ ]:
def Strait_area(massa, ro,  koef_str):
    """"Функция расчета площади пролива на неограниченное пространство
    Принмает массу, плотность вещества т/м3 и коэффициент разлития"""
    V_mass = massa / ro
    S = V_mass * koef_str
    return S

##Интенсивность испарения

In [ ]:
def Evaporation_404():
    """"Интенсивность испарения для нефти 
    кг/м2час????"""
    M = 200     #  Молярная масса нефти по расчетной Антона
    P_n = 7.840
    W = 10 ** -6 * sqrt(M) * P_n
    return W 

##Частота иницирующего события (test)

In [ ]:
df_freq['полная'][0]

5e-06

In [ ]:
def frequencies_pipline(diam):
  result = {'полная': '-' ,'частичная': '-'  }
  if diam < 75:
    result['полная'] = df_freq['полная'][0]
    result['частичная'] = df_freq['частичная'][0]
  elif 75 <= diam <= 150:
    result['полная'] = df_freq['полная'][1]
    result['частичная'] = df_freq['частичная'][1]    
  elif diam > 150:
    result['полная'] = df_freq['полная'][2]
    result['частичная'] = df_freq['частичная'][2] 
  else:
    return 'ERROR'

  return result 



##Частота реализации сценариев

In [ ]:
def scenarios():
  pass


##Классификация режимов сгорания облака

In [ ]:
def classification_cloud():
  pass 

##Функции округления (test)

In [ ]:
def round_to_n_decimals(x, n):
    if x == 0:
        return 0
    else:
        digits = int(math.log10(abs(x)))
        return round(x, -digits+n-1)

In [ ]:
def round_to_significant_figures(x, n):
    if x == 0:
        return 0
    else:
        return round(x, -int(math.floor(math.log10(abs(x))) + (n - 1)))

#Функции расчета зон поражения

##Взрыв топливно-воздушной смеси (ТВС)

In [ ]:
def TVS_404(M_t):
    """ Функция расчета взрыва ТВС
    принимает массу испарившихся веществ
    """
    result = {'100': '-','53': '-', '28': '-', '12': '-','5': '-','3': '-'}
    betta = 1.14   # коэффициент для нефти
    E = M_t * 44 * 10 ** 6 * betta      # Удельное энерговыделение
    u = 43 * M_t ** (1/6)          # дефлаграция скорости фронта пламени для 5 класса
    x = 0.1
    while True:
        Rx = x / ((E / P_0) ** (1/3))
        if Rx < 0.34:
            Px1 = (u ** 2 / 340 ** 2) * (6/7) * (0.83/0.34 - 0.14/0.34 ** 2)
        else:
            Px1 = (u ** 2 / 340 ** 2) * (6/7) * (0.83/Rx - 0.14/Rx ** 2)
        
        delta_P = Px1 * 101
        if round(delta_P,2) == 100.0:
          result['100'] = round(x,2)
        elif round(delta_P,2) == 53.0:
          result['53'] = round(x,2)
        elif round(delta_P,2) == 28.0:
          result['28'] = round(x,2)
        elif round(delta_P,2) == 12.0:
          result['12'] = round(x,2)   
        elif round(delta_P,2) == 5.0:
          result['5'] = round(x,2)        
        elif round(delta_P,2) == 3.0:
          result['3'] = round(x,2) 
          return result 
          break 
        elif delta_P < 1:
          return result 
          break
        x += 0.1

In [ ]:
TVS_404(500)

{'100': '-', '53': '-', '28': '-', '12': 31.9, '5': 102.9, '3': 179.6}

##Пожар-вспышка

In [ ]:
def fire_flash_404(m, ro_g = 10, C_nkpr = 1.1):
    """" Модуль расчета пожара вспышки.
    Принимает массу, плотность испарившейся нефти, Нижний концетрационный предел нефти"""
    ro_g = 10       # плотность ГГ нефти
    C_nkpr = 1.1        # нижний концентрационный предел распространения пламени нефти
    R_f = 1.2 * 7.8 * (m / (ro_g * C_nkpr))** 0.33
    return R_f

##Пожар пролива

In [ ]:
def StraitFire_404(S):
    """" Функция для расчета пожара пролива (!!! пока только нефть).
    В тестовом варианте принимает значение площади пролива S. 
    Следует дополнить следующими входными данными:
    - вещество (молярная масса, плотность насыщенных паров, плотность )
    - скорость ветра
    - учесть в функции разный расчет Ef для других видов веществ
    """
    result = {'10,5': '-', '7,0': '-', '1,4': '-'}
    d = sqrt(4 * S / pi)    #эффективный диаметр пролива
    m_u = 0.04          #удельная скорость выгорания топлива
    ro_parov = 218/22.4     #плотность насыщенных паров при испарении
    p_okr_pro = 1.2250      #плотность окружающего воздуха 
    w_0 = 0             #скорость ветра
    Ef = 140 + exp(-0.12 * d) + 20 * (1 - exp(-0.12 * d))       # среднеповерхностная интенсивность теплового излучения пламени
    theta = 0       # угол отклонения пламени от вертикали под действием ветра. (принимается равной 0, если не учитываем ветер)
    u_n = w_0 / np.cbrt(m_u * 9.81 * d / ro_parov)
    if u_n < 1:
        theta = 0
    else:
        theta = u_n ** (-0.5)
    theta = 0 
    
    if u_n >= 1:
        L = 55 * d * (m_u / (p_okr_pro * sqrt(9.81 * d)))** 0.67 * u_n ** 0.21
    else:
        L = 42 * d * (m_u / (p_okr_pro * sqrt(9.81 * d)))** 0.61
    x = 1
    while True:
        a = 2 * L / d
        b = 2 * x / d
        A = sqrt((a**2 + (b + 1)**2 - 2 * a * (b + 1) * sin(theta)))
        B = sqrt((a**2 + (b - 1)**2 - 2 * a * (b - 1) * sin(theta)))
        C = sqrt(1 + (b**2 - 1) * cos(theta)**2)
        D = abs((b - 1) / (b + 1)) ** 0.5
        E = a * cos(theta) / (b - a * sin(theta))
        F = abs(b**2 - 1) ** 0.5
        F_v = 1/pi * ( - E * atan(D) + E * ((a**2 + (b+1)**2 - 2*b * (1 + a * sin(theta)))/ (A * B)) * atan(A * D / B) + cos(theta)/C * (atan((a * b - F**2 * sin(theta)) / (F * C)) * atan(F**2 * sin(theta)/ (F * C))))
        F_n = 1/pi * (atan(1/D) + sin(theta)/C * (atan((a * b - F**2 * sin(theta)) / (F * C) ) + atan(F**2 * sin(theta) / (F * C))) - ((a**2 + (b+1)**2 - 2 *(b+1+a*b*sin(theta)))/(A * B)) * atan(A*D/B))
        F_q = sqrt(F_v**2 + F_n**2)
        ta = exp(-7 * 10 ** -4 * (x - 0.5 * d))
        q = Ef * F_q * ta
        if round(q, 2) == 10.5:
          result['10,5'] = round(x, 2)
        elif round(q) == 7:
          result['7,0'] = round(x, 2)
        elif round(q, 2) == 1.4:
          result['1,4'] = round(x, 2)
          return result 
          break 

        if q < 1:
          return result
          break
        x += 0.1


#Чтение и обработка данных из Excel

##считываем данные из заполненной таблицы Excel

In [ ]:
!gdown --id 1_Oa-FbrY7uSCpUt023fqL2CwtG1PlVEa

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_Oa-FbrY7uSCpUt023fqL2CwtG1PlVEa
To: /content/test.xlsx
100% 23.6k/23.6k [00:00<00:00, 30.4MB/s]


In [ ]:
df = pd.read_excel('test.xlsx', index_col='№')

In [ ]:
df.head()

,наименование оборудования,Тип оборудования,Вещество,Тип содержимого,"Количество опасного вещества, т",Температура,давление,"Интенсивность прокачки, м3/сут",время перекрытия,"Протяженность, м",диаметр,толщина стенки,Коэффициент подстил поверхности,Класс загроможденности пространства,Блок,№ в блоке
№,,,,,,,,,,,,,,,,
1,"Нефтесборный коллектор ""Куст № 2 - т.в. Куста ...",труба,нефть,ЛВЖ,NaN,-20..20,4,26.0,300,270,219,8,5,4,Газопровод внешнего транспорта,1
2,Нефтепровод от куста №1 до УПН Снежного нефтян...,труба,нефть,ЛВЖ,NaN,-20..20,4,142.0,300,3050,114,6,5,4,Газопровод внешнего транспорта,2
3,Нефтепровод от куста №1 до УПН Снежного нефтян...,труба,нефть,ЛВЖ,NaN,-20..20,4,142.0,300,706,159,6,5,4,Газопровод внешнего транспорта,3
4,Нефтепровод от куста №1 до УПН Снежного нефтян...,труба,нефть,ЛВЖ,NaN,-20..20,4,142.0,300,355,219,8,5,4,Газопровод внешнего транспорта,4
5,Нефтесборные сети куст №3 - т. вр. Куста № 1 - 1,труба,нефть,ЛВЖ,NaN,-20..20,4,95.1,300,3100,159,6,5,4,Газопровод внешнего транспорта,5


##Расчитываем объем и массу по каждому участку трубопровода

In [ ]:
df['объем'] = round(Volume_pipeline(df['Протяженность, м'], df['диаметр'], df['толщина стенки']), 2)

In [ ]:
df.head()

,наименование оборудования,Тип оборудования,Вещество,Тип содержимого,"Количество опасного вещества, т",Температура,давление,"Интенсивность прокачки, м3/сут",время перекрытия,"Протяженность, м",диаметр,толщина стенки,Коэффициент подстил поверхности,Класс загроможденности пространства,Блок,№ в блоке,объем
№,,,,,,,,,,,,,,,,,
1,"Нефтесборный коллектор ""Куст № 2 - т.в. Куста ...",труба,нефть,ЛВЖ,NaN,-20..20,4,26.0,300,270,219,8,5,4,Газопровод внешнего транспорта,1,8.74
2,Нефтепровод от куста №1 до УПН Снежного нефтян...,труба,нефть,ЛВЖ,NaN,-20..20,4,142.0,300,3050,114,6,5,4,Газопровод внешнего транспорта,2,24.92
3,Нефтепровод от куста №1 до УПН Снежного нефтян...,труба,нефть,ЛВЖ,NaN,-20..20,4,142.0,300,706,159,6,5,4,Газопровод внешнего транспорта,3,11.98
4,Нефтепровод от куста №1 до УПН Снежного нефтян...,труба,нефть,ЛВЖ,NaN,-20..20,4,142.0,300,355,219,8,5,4,Газопровод внешнего транспорта,4,11.49
5,Нефтесборные сети куст №3 - т. вр. Куста № 1 - 1,труба,нефть,ЛВЖ,NaN,-20..20,4,95.1,300,3100,159,6,5,4,Газопровод внешнего транспорта,5,52.61


In [ ]:
df['масса_в_участке'] = round(df['объем'] * ro, 2)

##Масса вещества участвующего в аварии и площади пролива

In [ ]:
preparation_df = pd.DataFrame(columns=['полная', 'частичная','площадь полная', 'площадь частичная', 'облако', 'твс'])

In [ ]:
preparation_df.insert(0,'наименование оборудования', df['наименование оборудования'])

In [ ]:
preparation_df['полная'] = df['масса_в_участке'] + df['Интенсивность прокачки, м3/сут']/3600 * ro  * df['время перекрытия']

In [ ]:
preparation_df['площадь полная'] = Strait_area(preparation_df['полная'], ro, df['Коэффициент подстил поверхности'])

In [ ]:
preparation_df['облако'] = Evaporation_404() * 3600 * preparation_df['площадь полная']

In [ ]:
preparation_df['твс'] = preparation_df['облако'] * 0.1

In [ ]:
preparation_df

,наименование оборудования,полная,частичная,площадь полная,площадь частичная,облако,твс
№,,,,,,,
1,"Нефтесборный коллектор ""Куст № 2 - т.в. Куста ...",9.708333,NaN,54.541199,NaN,21.769990,2.176999
2,Нефтепровод от куста №1 до УПН Снежного нефтян...,32.711667,NaN,183.773408,NaN,73.352721,7.335272
3,Нефтепровод от куста №1 до УПН Снежного нефтян...,21.191667,NaN,119.054307,NaN,47.520245,4.752025
4,Нефтепровод от куста №1 до УПН Снежного нефтян...,20.761667,NaN,116.638577,NaN,46.556012,4.655601
5,Нефтесборные сети куст №3 - т. вр. Куста № 1 - 1,53.873250,NaN,302.658708,NaN,120.805508,12.080551
6,Нефтесборные сети куст №3 - т. вр. Куста № 1 - 2,122.273250,NaN,686.928371,NaN,274.185835,27.418584
7,Внутрипромысловый нефтепровод Куст № 3 - Куст № 4,33.833250,NaN,190.074438,NaN,75.867763,7.586776
8,Внутрипромысловый нефтерповод Куст №5 - Куст №3,8.503250,NaN,47.771067,NaN,19.067709,1.906771
9,Внутрипромысловый нефтепровод Куст №6 - Вр. Ку...,12.873250,NaN,72.321629,NaN,28.867007,2.886701


##С-2 Сценарий пожар-пролива

In [ ]:
scenario_c2_df = pd.DataFrame(columns=['10,5', '7,0','1,4'])

In [ ]:
scenario_c2_df.insert(0,'наименование оборудования', df['наименование оборудования'] )

In [ ]:
scenario_c2_df.insert(1,'площадь пожара', preparation_df['площадь полная'] )

In [ ]:
for index, row in scenario_c2_df.iterrows():
  result = StraitFire_404(row[1])
  for key, value in result.items():
    scenario_c2_df.loc[index, key] = value

In [ ]:
scenario_c2_df

,наименование оборудования,площадь пожара,"10,5","7,0","1,4"
№,,,,,
1,"Нефтесборный коллектор ""Куст № 2 - т.в. Куста ...",54.541199,-,19.6,42.2
2,Нефтепровод от куста №1 до УПН Снежного нефтян...,183.773408,-,34.2,71.6
3,Нефтепровод от куста №1 до УПН Снежного нефтян...,119.054307,-,28.1,59.3
4,Нефтепровод от куста №1 до УПН Снежного нефтян...,116.638577,-,27.8,58.8
5,Нефтесборные сети куст №3 - т. вр. Куста № 1 - 1,302.658708,-,42.8,88.7
6,Нефтесборные сети куст №3 - т. вр. Куста № 1 - 2,686.928371,-,61.7,125.6
7,Внутрипромысловый нефтепровод Куст № 3 - Куст № 4,190.074438,-,34.7,72.7
8,Внутрипромысловый нефтерповод Куст №5 - Куст №3,47.771067,-,18.5,39.8
9,Внутрипромысловый нефтепровод Куст №6 - Вр. Ку...,72.321629,-,22.4,47.8


##С-3 Сценарий пожар вспышка

In [ ]:
scenario_c3_df = pd.DataFrame(columns=['вспышка'])

In [ ]:
scenario_c3_df.insert(0,'наименование оборудования', df['наименование оборудования'] )

In [ ]:
scenario_c3_df['вспышка'] = fire_flash_404(preparation_df['облако'])

##С-4 Сценарий Взрыв ТВС

In [ ]:
scenario_c4_df = pd.DataFrame(columns=['100', '53','28', '12', '5', '3'])

In [ ]:
scenario_c4_df.insert(0,'наименование оборудования', df['наименование оборудования'] )

In [ ]:
scenario_c4_df.insert(1,'масса', preparation_df['твс'] )

In [ ]:
for index, row in scenario_c4_df.iterrows():
  result = TVS_404(row[1])
  for key, value in result.items():
    scenario_c4_df.loc[index, key] = value

In [ ]:
scenario_c4_df

,наименование оборудования,масса,100,53,28,12,5,3
№,,,,,,,,
1,"Нефтесборный коллектор ""Куст № 2 - т.в. Куста ...",2.176999,-,-,-,-,-,-
2,Нефтепровод от куста №1 до УПН Снежного нефтян...,7.335272,-,-,-,-,-,7.5
3,Нефтепровод от куста №1 до УПН Снежного нефтян...,4.752025,-,-,-,-,-,-
4,Нефтепровод от куста №1 до УПН Снежного нефтян...,4.655601,-,-,-,-,-,-
5,Нефтесборные сети куст №3 - т. вр. Куста № 1 - 1,12.080551,-,-,-,-,-,-
6,Нефтесборные сети куст №3 - т. вр. Куста № 1 - 2,27.418584,-,-,-,-,9.6,22.7
7,Внутрипромысловый нефтепровод Куст № 3 - Куст № 4,7.586776,-,-,-,-,-,-
8,Внутрипромысловый нефтерповод Куст №5 - Куст №3,1.906771,-,-,-,-,-,-
9,Внутрипромысловый нефтепровод Куст №6 - Вр. Ку...,2.886701,-,-,-,-,-,-


##Определение вероятности реализации событий

In [ ]:
frequencies_df = pd.DataFrame(columns=['полная','частичная',
                                       'С-1А(п)', 'С-1А(ч)', 'С-2', 'С-3', 'С-4' ]) 

In [ ]:
frequencies_df.insert(0,'наименование оборудования', df['наименование оборудования'] )

Расчитываем вероятность возникновения иницирующих событий с учетом длины трубопроводов

In [ ]:
for index, row in df.iterrows():
  result = frequencies_pipline(row['диаметр']) 
  for key, value in result.items():
    frequencies_df.loc[index, key] = value * row['Протяженность, м']

Расчитываем вероятность возникновения различных сценариев

In [ ]:
for index, row in frequencies_df.iterrows():
  
   frequencies_df.loc[index, 'С-1А(п)'] = row['полная'] * scenario_lvz['полная']['пролив']
   frequencies_df.loc[index, 'С-1А(ч)'] = row['частичная'] * scenario_lvz['частичная']['пролив']
   frequencies_df.loc[index, 'С-2'] = row['полная'] * scenario_lvz['полная']['пожар']
   frequencies_df.loc[index, 'С-3'] = row['полная'] * scenario_lvz['полная']['вспышка']
   frequencies_df.loc[index, 'С-4'] = row['полная'] * scenario_lvz['полная']['ТВС'] 

In [ ]:
frequencies_df

,наименование оборудования,полная,частичная,С-1А(п),С-1А(ч),С-2,С-3,С-4
№,,,,,,,,
1,"Нефтесборный коллектор ""Куст № 2 - т.в. Куста ...",0.000135,0.000027,0.000082,0.000025,0.000027,0.00001,0.000016
2,Нефтепровод от куста №1 до УПН Снежного нефтян...,0.0061,0.000915,0.003709,0.000851,0.00122,0.000468,0.000703
3,Нефтепровод от куста №1 до УПН Снежного нефтян...,0.000353,0.000071,0.000215,0.000066,0.000071,0.000027,0.000041
4,Нефтепровод от куста №1 до УПН Снежного нефтян...,0.000178,0.000035,0.000108,0.000033,0.000036,0.000014,0.00002
5,Нефтесборные сети куст №3 - т. вр. Куста № 1 - 1,0.00155,0.00031,0.000942,0.000288,0.00031,0.000119,0.000179
6,Нефтесборные сети куст №3 - т. вр. Куста № 1 - 2,0.002,0.0004,0.001216,0.000372,0.0004,0.000154,0.00023
7,Внутрипромысловый нефтепровод Куст № 3 - Куст № 4,0.000886,0.000177,0.000539,0.000165,0.000177,0.000068,0.000102
8,Внутрипромысловый нефтерповод Куст №5 - Куст №3,0.000824,0.000124,0.000501,0.000115,0.000165,0.000063,0.000095
9,Внутрипромысловый нефтепровод Куст №6 - Вр. Ку...,0.000203,0.000041,0.000124,0.000038,0.000041,0.000016,0.000023


#Заполнение данных в Excel

In [ ]:
wb = load_workbook('test.xlsx')

In [ ]:
wb_rasp = wb['Rasp']

In [ ]:
for i, r in df.iterrows():
  wb_rasp.cell(row = i + 3, column = 1, value = df['Блок'][i])
  wb_rasp.cell(row = i + 3, column = 2, value = f'''{df['наименование оборудования'][i]}, {df['Вещество'][i]}''')
  wb_rasp.cell(row = i + 3, column = 3, value = f'''{df['Протяженность, м'][i]}, м''')
  wb_rasp.cell(row = i + 3, column = 4, value = f'''{str(df['масса_в_участке'][i]).replace('.', ',')}''')
  wb_rasp.cell(row = i + 3, column = 4).number_format = '0,00'
  wb_rasp.cell(row = i + 3, column = 6, value = f'''Жидкость''')
  wb_rasp.cell(row = i + 3, column = 7, value = f'''{df['давление'][i]}''')
  wb_rasp.cell(row = i + 3, column = 8, value = f'''{df['Температура'][i]}''')  
wb_rasp.cell(row = 4, column = 5, value = str(df.groupby("Блок")["масса_в_участке"].sum()[0]).replace('.', ','))
wb_rasp.merge_cells(start_row=4, start_column=5, end_row=12, end_column=5)
#итог
wb_rasp.cell(row = 13, column = 1, value = 'Итого опасного вещества - Нефть,т')
wb_rasp.cell(row = 13, column = 5, value = str(df.groupby("Блок")["масса_в_участке"].sum()[0]).replace('.', ','))
wb_rasp.merge_cells(start_row=13, start_column=1, end_row=13, end_column=4)
wb_rasp.merge_cells(start_row=13, start_column=5, end_row=13, end_column=8)
wb_rasp.cell(row = 14, column = 1, value = 'из них - в сосудах (аппаратах), т')
wb_rasp.cell(row = 14, column = 5, value = '0')
wb_rasp.merge_cells(start_row=14, start_column=1, end_row=14, end_column=4)
wb_rasp.merge_cells(start_row=14, start_column=5, end_row=14, end_column=8)
wb_rasp.cell(row = 15, column = 1, value = 'в трубопроводах, т')
wb_rasp.cell(row = 15, column = 5, value = str(df.groupby("Блок")["масса_в_участке"].sum()[0]).replace('.', ','))
wb_rasp.merge_cells(start_row=15, start_column=1, end_row=15, end_column=4)
wb_rasp.merge_cells(start_row=15, start_column=5, end_row=15, end_column=8)

In [ ]:
wb_mass = wb['Masses']

In [ ]:
for i in wb_rasp.iter_rows(min_col=1, min_row=1, values_only=True):
  print(i)


('Технологический блок, оборудование', None, None, 'Количество опасного вещества, т', None, 'Физические условия содержания опасного вещества', None, None)
('наименование технологического участка (блока)', 'наименование трубопровода, опасное вещество', 'длина, м / количество   шт', 'в единице оборудования (трубопроводе)', 'в блоке', 'агрегатное состояние', 'давление, МПа', 'температура, °С')
(1, 2, 3, 4, 5, 6, 7, 8)
('Газопровод внешнего транспорта', 'Нефтесборный коллектор "Куст № 2 - т.в. Куста № 2", нефть', '270, м', '7,78', '246,94', 'Жидкость', '4', '-20..20')
('Газопровод внешнего транспорта', 'Нефтепровод от куста №1 до УПН Снежного нефтяного месторождения - 1, нефть', '3050, м', '22,18', None, 'Жидкость', '4', '-20..20')
('Газопровод внешнего транспорта', 'Нефтепровод от куста №1 до УПН Снежного нефтяного месторождения - 2, нефть', '706, м', '10,66', None, 'Жидкость', '4', '-20..20')
('Газопровод внешнего транспорта', 'Нефтепровод от куста №1 до УПН Снежного нефтяного месторожде

In [ ]:
#wb.save('/content/drive/MyDrive/Work, Study, Life/Analyst/Pipeline_method/result.xlsx')